# Python DS-ao-Dev

## Bussiness Problem



**Star Jeans Company**

- Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um modelo de negócio do tipo E-commerce.

- A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação baixo e escalar a medida que forem conseguindo clientes.

- Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e o material para a fabricação de cada peça.

- Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes perguntas: ]
    1. Qual o melhor preço de venda para as calças? 
    2. Quantos tipos de calças e suas cores para o produto inicial? 
    3. Quais as matérias-prima necessárias para confeccionar as calças?
    
- As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

## Solution Planning (Input-Output-Tasks)





**Bussiness Question**

- Which is the best price for jeans?

1. Input:
    1. Fonte de dados
        - Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
        - Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans
    2. Ferramentas
        - Python 3.8.0
        - Bibliotecas de Webscrapping ( BS4, Selenium )
        - PyCharm
        - Jupyter Notebook ( Analise e prototipagens )
        - Crontjob, Airflow
        - Streamlit
    
2. Output:
    1. A resposta para a pergunta.
        - Mediana dos preços dos concorrents.
    2. Formato da entrega
        - Tabela ou gráfico
    3. Local da entrega
        - App Streamlit
    
3. Tasks:
    1. Passo a passso para construir o cálculo da mediana ou média
        - Realizar o calculo da mediana sobre o produto, tipo e cor
    2. Definir o formato da entrega ( Visualização, Tabela, Frase )
        - Gráfico de barras com a mediana dos preço dos produtos, por tipo e cor dos últimos 30 dia
        - Tabela com as seguintes colunas: id | product_name | product_type | product_color | produ
        - Definição do schema: Colunas e seu tipo
        - Definição a infraestrutura de armazenamento ( SQLITE3 )
        - Design do ETL ( Scripts de Extração, Transformação e Carga )
        - Planejamento de Agendamento dos scripts ( dependencias entre os scripts )
        - Fazer as visualizações
        - Entrega do produto final
    3. Decidir o local de entrega ( PowerBi, Telegram, Email, Streamlit, Intranet ),
        - App com Streamlit


## Bussiness Models



“Como você planeja ganhar dinheiro”, Michael Lewis

“Um modelo de negócio descreve a lógica de criação, entrega e captura de valor por
parte de uma organização”, Alexander Osterwalder

- E-commerce:
    1. Faturamento: Vendas de um produto.
    2. Exemplo: Lojas Riachuelo, Submarino, Magazine Luiza, etc
        
- Software AS a Service ( SaaS ):
    1. Faturamento: Assinatura mensal/anual de utilização ou por usuário.
    2. Exemplo: Looker, Asana, Gmail, Salesforce.
    
- Serviço:
    1. Faturamento: Prestação de serviço por tempo ou projeto.
    2. Exemplo: Sul América, Porto Seguro, Mapfre.
    
- Mobile App:
    1. Faturamento: Venda de upgrades.
    2. Exemplo: Wildlife, Ubisoft, Games Mobile.
    
- Media Site:
    1. Faturamento: Cobrança por clicks ou visualizações de um determinado anúncio.
    2. Exemplo: Facebook, Google, UOL, G1, etc.
    
- Marketplace:
    1. Faturamento: Taxa sobre a transação entre o passageiro e o motorista.
    2. Exemplo: Uber, Ifood, 99, Elo7, Submarino.


## E-commerce Metrics

- **Growth Metrics**:
    1. Porcentagem do Marketshare
    2. Número de Clientes Novos
- **Revenue Metrics**:
    1. Número de Vendas
    2. Ticket Médio
    3. LTV ( Long Time Value )
    4. Recência Média
    5. Basket Size Médio
    6. Markup médio
- **Cost Metrics**:
    1. CAC ( Custo de aquisição de Clientes )
    2. Desconto médio
    3. Custo de Produção
    4. Taxa de devolução
    5. Custos Fixos ( Folha de pagamento, escritório, softwares )
    6. Impostos

# Imports

In [51]:
import re
import sqlite3
import requests

import pandas as pd
import numpy  as np

from bs4        import BeautifulSoup
from datetime   import datetime

from sqlalchemy import create_engine

# Data Gathering

## Showcase

In [2]:
# parameters
headers = {'user-agent': 'my-app/0.0.1'}

# URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

#request to URL
page = requests.get(url, headers = headers)

# BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

# ============== Product Data ===============
#website showcase
products = soup.find('ul', class_= 'products-listing small') #find retorna apenas 1 elemento, pois temos apenas 1 UL, uma vitrine, find_all retorna lista

#list comprehension to get all products id and products category from the first page of 
product_list = products.find_all('article', class_ = 'hm-product-item')

# product id
product_id = [p.get('data-articlecode') for p in product_list]

# product_category
product_category = [p.get('data-category') for p in product_list]

# product name
product_list = products.find_all('a', class_ = 'link')
product_name = [p.get_text() for p in product_list]

#price
product_list = soup.find_all('span', class_ = 'price regular')
product_price = [p.get_text() for p in product_list]

data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

#scrapy time 
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,0979945001,men_jeans_loose,Loose Jeans,$ 29.99,2022-02-21 19:15:49
1,1008549001,men_jeans_regular,Regular Jeans,$ 19.99,2022-02-21 19:15:49
2,1013317006,men_jeans_regular,Hybrid Regular Tapered Joggers,$ 39.99,2022-02-21 19:15:49
3,1013317002,men_jeans_regular,Hybrid Regular Tapered Joggers,$ 39.99,2022-02-21 19:15:49
4,1008549004,men_jeans_regular,Regular Jeans,$ 19.99,2022-02-21 19:15:49


In [3]:
data.shape

(36, 5)

## By products

In [99]:
# empty  dataframe
df_compositions = pd.DataFrame()

# unique columns for all products
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size', 'More sustainable materials']
df_pattern = pd.DataFrame(columns = cols)

for i in range(len(data)):
    # API Request
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    print('Product: {}'.format(url))
    
    page = requests.get(url, headers = headers)
    
    # Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # ==================== color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature active' ) + soup.find_all( 'a', class_='filter-option miniature' )
    
    # color name
    color_name = [p.get( 'data-color' ) for p in product_list]
    
    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range(len(df_color)): #go through all colors and collect each composition 
        # API Request
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
    
        print('Color: {}'.format(url))

        page = requests.get(url, headers = headers)

        # Beautiful Soup object
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # ============ Product Name =============roduct_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_name = soup.find_all('h1', class_ = 'primary product-item-headline')
#       product_name = [ p.get_text() for p in product_name]
        product_name = product_name[0].get_text()
    
        # ============ Product Price =============
        product_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_price = re.findall(r'\d+.?\d+', product_price[0].get_text())[0]     
        
#         df_product_name_price = pd.DataFrame([product_name, product_price]).T
        
        # =================== composition =====================
        product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
        product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # create composition dataframe
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row
        df_composition = df_composition.iloc[1:].fillna(method = 'ffill')

        # remove pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket lining: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Shell: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Lining: ', '', regex = True)

        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        #rename columns
        df_composition.columns = ['product_id','composition','fit','product_safety', 'size', 'sustainable_materials']

        #create columns product name and product price
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price
        
        #keep new columns if it shows up
        aux = aux + df_composition.columns.tolist() #to guarantee we have all columns of composition unique values

        # merge data color + composition
        df_composition = pd.merge( df_composition, df_color, how = 'left', on = 'product_id')

        # all products
        df_compositions = pd.concat([df_compositions, df_composition], axis = 0)
    
# # Join Showroom data + details
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )

#scrapy time 
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

Product: https://www2.hm.com/en_us/productpage.0979945001.html
Color: https://www2.hm.com/en_us/productpage.0979945001.html


IndexError: single positional indexer is out-of-bounds

In [88]:
product_composition

[[]]

In [102]:
a = soup.find_all( 'div', class_='ProductAttributesList-module--descriptionListItem__3vUL2' )
a

[]

In [100]:
product_composition_list

[]

In [97]:
product_composition_list

[<div class="content pdp-text pdp-content">
 <hm-product-description id="js-product-description"></hm-product-description>
 <section class="review-answers selected-rating" id="js-selected-rating"></section>
 </div>]

In [72]:
product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

In [73]:
product_composition

[[]]

In [63]:
df_color.head()

,product_id,color_name
0,0979945001,Light denim blue
1,0979945002,Denim blue
2,0979945003,Black


In [53]:
df_composition.head()

,product_id,composition,fit,product_safety,size,sustainable_materials,product_name,product_price,color_name
0,0875105023,"Polyester 65%, Cotton 35%",Relaxed fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Relaxed Jeans,29.99,Black
1,0875105023,Cotton 100%,Relaxed fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Relaxed Jeans,29.99,Black


In [61]:
df_composition.head()

""


In [59]:
df_composition.iloc[1
                   ]

product_id                                                   0875105023
composition                                                 Cotton 100%
fit                                                         Relaxed fit
product_safety                                                      NaN
size                     The model is 189cm/6'2" and wears a size 32/32
sustainable_materials                                               NaN
product_name                            \n\t\t\t\t\t\t\t  Relaxed Jeans
product_price                                                     29.99
color_name                                                        Black
Name: 1, dtype: object

In [16]:
len(df_compositions['product_id'].unique())

96

# Data Cleaning

In [33]:
#product id
df_data = df_compositions.dropna(subset = ['product_id'])

#product name

df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace('\n', ''))
df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace('\t', ''))
df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace('  ', ''))
df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

#product price
df_data['product_price'] = df_data['product_price'].astype(float)

#color name
df_data['color_name'] = df_data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

# #fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# #====  size  ======
# #size number
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
df_data['size_number'] = df_data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x) #group(0) locates the whole match expression

# #size model 
df_data['size_model'] = df_data['size'].str.extract('(\d+/\\d+)') #.str to vectorize the lines, .extracts cant be applied in the whole column

# # #product safety

# =================== sustainable materials ============
df_data['sustainable_materials'] = df_data['sustainable_materials'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# df2 = df_data['sustainable_materials'].str.split()

#recycled cotton / recycled polyester
# creating an empty dataframe as reference to organize the wnanted columns
# thenconcatenete with the main dataframe, but it has to have the same lenght as 'data' dataframe

df2 = df_data[['sustainable_materials']].reset_index(drop=True)
df_ref2 = pd.DataFrame(index = np.arange(len(df_data)), columns = ['recycled_cotton', 'recycled_polyester'])

# ------------ recycled cotton ------------
df_recycled_cotton = df2.loc[df2['sustainable_materials'].str.contains('recycled_cotton', na = True), 'sustainable_materials']
df_recycled_cotton.name = 'recycled_cotton'

df_ref2 = pd.concat([df_ref2, df_recycled_cotton], axis = 1)
df_ref2 = df_ref2.iloc[:, ~df_ref2.columns.duplicated(keep='last')]

# ------------recycled polyester -------------
df_recycled_polyester = df2.loc[df2['sustainable_materials'].str.contains('recycled_polyester', na = True), 'sustainable_materials']
df_recycled_polyester.name = 'recycled_polyester'

df_ref2 = pd.concat([df_ref2, df_recycled_polyester], axis = 1)
df_ref2 = df_ref2.iloc[:, ~df_ref2.columns.duplicated(keep='last')]

# ====================  composition =====================
#break composition by comma
df1 = df_data['composition'].str.split(',', expand = True).reset_index(drop=True)

# cotton / polyester / spandex / 
# creating empty dataframe as reference to organize the wanted columns and 
# then concatanete with the main dataframe, but it has to have the same lenght as 'data' dataframe

df_ref = pd.DataFrame(index = np.arange(len(df_data)), columns = ['cotton', 'polyester', 'spandex'])

# --------------- cotton ----------------
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na = True), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na = True), 1]
df_cotton_1.name = 'cotton'

#combine
df_cotton = df_cotton_0.combine_first(df_cotton_1)

df_ref = pd.concat([df_ref, df_cotton], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep = 'last')] #exclue colunas duplicadas, mantem as diferentes
# df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# -------------- polyester ----------------
df_polyester_0 = df1.loc[df1[0].str.contains('Polyester', na = True), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains('Polyester', na = True), 1]
df_polyester_1.name = 'polyester'

# combine
df_polyester = df_polyester_0.combine_first(df_polyester_1)

# add to reference dataframe
df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')] #delet duplicated columns and keep the different ones

# -------------- spandex ------------------
df_spandex_1 = df1.loc[df1[1].str.contains('Spandex', na = True), 1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains('Spandex', na = True), 2]
df_spandex_2.name = 'spandex'

# combine
df_spandex = df_spandex_1.combine_first(df_spandex_2)

df_ref = pd.concat([df_ref, df_spandex], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# # ------------- elasterell -----------
# df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na = True), 1]
# df_elasterell.name = 'elasterell'

# df_ref = pd.concat([df_ref, df_elasterell], axis = 1)
# df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# combine the two dataframe reference
df_ref_final = pd.concat([df_ref, df_ref2], axis = 1)

# add product_id in the aux dataframe
df_aux = pd.concat([df_data['product_id'].reset_index(drop = True), df_ref_final], axis = 1)

# formatt composition data
df_aux['cotton'] = df_aux['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['polyester'] = df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['spandex'] = df_aux['spandex'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['recycled_cotton'] = df_aux['recycled_cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['recycled_polyester'] = df_aux['recycled_polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

# final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna( 0 ) #choose the highest value between lines
df_data = pd.merge(df_data, df_aux, on = 'product_id', how = 'left')

#d rop columns
df_data = df_data.drop(columns = ['size', 'product_safety','composition','sustainable_materials'], axis = 1)

#drop duplicates

df_data = df_data.drop_duplicates()

df_data.shape

(96, 15)

In [34]:
df_data.head()

,product_id,fit,product_name,product_price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex,recycled_cotton,recycled_polyester
0,0979945001,loose_fit,loose_jeans,29.99,light_denim_blue,0979945,001,2022-02-21 19:31:32,180,31/32,1.00,0.65,0.00,0.2,0.0
2,0979945002,loose_fit,loose_jeans,29.99,denim_blue,0979945,002,2022-02-21 19:31:32,NaN,NaN,1.00,0.65,0.00,0.2,0.0
4,0979945003,loose_fit,loose_jeans,29.99,black,0979945,003,2022-02-21 19:31:32,NaN,NaN,1.00,0.65,0.00,0.2,0.0
6,1008549001,regular_fit,regular_jeans,19.99,light_denim_blue,1008549,001,2022-02-21 19:31:32,NaN,NaN,0.98,0.65,0.02,0.2,0.0
8,1008549002,regular_fit,regular_jeans,19.99,denim_blue,1008549,002,2022-02-21 19:31:32,185,31/32,0.99,0.65,0.01,0.2,0.0


In [35]:
df_data.shape

(96, 15)

# Data Insert

In [39]:
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'fit',
    'product_price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'recycled_cotton',
    'recycled_polyester',
    'scrapy_datetime'

]]

In [45]:
# query_showroom_schema = """
#     CREATE TABLE VITRINE (
    
#         product_id          TEXT,
#         style_id            TEXT,
#         color_id            TEXT,
#         product_name        TEXT,
#         color_name          TEXT,
#         fit                 TEXT,
#         product_price       REAL,
#         size_number         TEXT,
#         size_model          TEXT,
#         cotton              REAL,
#         polyester           REAL,
#         spandex             REAL,
#         recycled_cotton     REAL,
#         recycled_polyester  REAL,
#         scrapy_datetime     TEXT
#     )

# """

In [46]:
#create table
conn = sqlite3.connect('database_hm.sqlite')
conn.execute( query_showroom_schema )
conn.commit()

In [48]:
# create database connection
conn = create_engine( 'sqlite:///database_hm.sqlite', echo = False)

# data insert
data_insert.to_sql('vitrine', con = conn, if_exists = 'append', index = False )

In [52]:
pwd

'/home/marxcerqueira/repos/python-ds-ao-dev'

In [50]:
# query = """
#     SELECT * FROM vitrine

# """
# df = pd.read_sql_query(query, conn)
# df.head()

,product_id,style_id,color_id,product_name,color_name,fit,product_price,size_number,size_model,cotton,polyester,spandex,recycled_cotton,recycled_polyester,scrapy_datetime
0,0979945001,0979945,001,loose_jeans,light_denim_blue,loose_fit,29.99,180,31/32,1.00,0.65,0.00,0.2,0.0,2022-02-21 19:31:32
1,0979945002,0979945,002,loose_jeans,denim_blue,loose_fit,29.99,None,None,1.00,0.65,0.00,0.2,0.0,2022-02-21 19:31:32
2,0979945003,0979945,003,loose_jeans,black,loose_fit,29.99,None,None,1.00,0.65,0.00,0.2,0.0,2022-02-21 19:31:32
3,1008549001,1008549,001,regular_jeans,light_denim_blue,regular_fit,19.99,None,None,0.98,0.65,0.02,0.2,0.0,2022-02-21 19:31:32
4,1008549002,1008549,002,regular_jeans,denim_blue,regular_fit,19.99,185,31/32,0.99,0.65,0.01,0.2,0.0,2022-02-21 19:31:32
